# PRO3515 - PFSP LS


## Dupla 9

### Nomes: 
### Lucas Hideki Takeuchi Okamura NUSP:  9274315
### Thales Arantes Kerche Nunes   NUSP: 10769372


Importando as bibliotecas necessárias

In [2]:
import xlwings as xw
import gurobipy as grb
import numpy as np

Definindo a classe Job que usaremos no algoritmo

In [6]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.p_total=sum(self.p)
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness
        self.bestPos=-1
    
    def __str__(self):
        return 'Job #{}, p = {}'.format(self.i, self.p)

### Reutilizamos a função disponibilizada pelo professor, com alguns ajustes para ler o número de máquinas

In [7]:
def leInst(sheet_num):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('B11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
#     print('n =',n)
#     print('m =',m)
#     print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i]
        jobs.append(Job(i,pi,0))
    
    return m,n,jobs

In [8]:
def gravaSched(i,jobs,Cmax):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',5]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5']
    plan.range('A11').value=tab

    print()
    print('Cmax (I('+str(i+1)+')): ', Cmax)

In [9]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

In [84]:
def schedGurobi(jobs: list, m: int = 5):
    
    n = len(jobs)
    mPFSP = grb.Model(name="Permutation Flow Shop Problem")
    x = mPFSP.addVars(n, n, vtype=grb.GRB.BINARY, name='x')
    s = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb=0, name='s')
    c = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb=0, name='c')
    cmax = mPFSP.addVar(vtype=grb.GRB.CONTINUOUS, lb=0, name='cmax')
    mPFSP.update()

    mPFSP.addConstrs((s[j,k] - c[j,k] + grb.quicksum(jobs[i].p[k]*x[i,k] for i in range(n)) <= 0 for k in range(m) for j in range(n)), name='r1')
    mPFSP.addConstrs((s[j,k]-c[j-1,k]>=0 for k in range(m) for j in range(1, n)), name='r2')
    mPFSP.addConstrs((s[j,k]-c[j,k-1]>=0 for k in range(1, m) for j in range(n)), name='r3')
    mPFSP.addConstrs((grb.quicksum(x[i,j] for i in range(n)) == 1  for j in range(n)), name='r4')
    mPFSP.addConstrs((grb.quicksum(x[i,j] for j in range(n)) == 1  for i in range(n)), name='r5')
    mPFSP.addConstr(cmax == grb.max_(c[n-1,k] for k in range(m)))
    mPFSP.update()
    
    mPFSP.ModelSense = grb.GRB.MINIMIZE
    mPFSP.setObjective(cmax)
    mPFSP.update()
    
    mPFSP.optimize()
    runtime = mPFSP.Runtime
    CmaxFinal = mPFSP.objVal
    gap = mPFSP.MIPGap
    
    pos = np.reshape(mPFSP.getVars()[:n*n], (n, n))
    for i in range(n):
        for j in range(n):
            if pos[i, j].x > 0.999:
                jobs[i].bestPos = j
                break

    print([job.i+1 for job in jobs])
    jobs.sort(key=lambda job: job.bestPos)
    print(calcCmax(m, jobs))
    if calcCmax(m, jobs) == int(c[n-1,m-1].x):
        print("OK!")
    
    return jobs, CmaxFinal, runtime, gap, mPFSP

## Principal

Caso base pra comprovar o funcionamento do algoritmo implementado

In [85]:
tab1 = [[16, 18, 12],[14,10,11],[13,20,15],[19,15,19],[15,16,16]]
jobs=[]
for i in range(len(tab1)):
    pi=tab1[i]
    jobs.append(Job(i,pi,0))

jobs, Cmax, runtime, gap, model = schedGurobi(jobs, 3)

print([job.i+1 for job in jobs])
print([job.p for job in jobs])
print('Cmax = ', Cmax)
print('Runtime = ', runtime)
print('Gap = ', gap)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 47 rows, 56 columns and 199 nonzeros
Model fingerprint: 0x9f2b520a
Model has 1 general constraint
Variable types: 31 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 10 rows and 14 columns
Presolve time: 0.00s
Presolved: 37 rows, 42 columns, 151 nonzeros
Variable types: 15 continuous, 27 integer (25 binary)

Root relaxation: objective 8.700000e+01, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      87.0000000   87.00000  0.00%     -    0s

Explored 0 nodes (19 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 87 

Optimal solution found (t

In [86]:
model.getVars()

[<gurobi.Var x[0,0] (value -0.0)>,
 <gurobi.Var x[0,1] (value -0.0)>,
 <gurobi.Var x[0,2] (value 1.0)>,
 <gurobi.Var x[0,3] (value -0.0)>,
 <gurobi.Var x[0,4] (value 0.0)>,
 <gurobi.Var x[1,0] (value -0.0)>,
 <gurobi.Var x[1,1] (value 1.0)>,
 <gurobi.Var x[1,2] (value 0.0)>,
 <gurobi.Var x[1,3] (value -0.0)>,
 <gurobi.Var x[1,4] (value -0.0)>,
 <gurobi.Var x[2,0] (value 1.0)>,
 <gurobi.Var x[2,1] (value -0.0)>,
 <gurobi.Var x[2,2] (value -0.0)>,
 <gurobi.Var x[2,3] (value -0.0)>,
 <gurobi.Var x[2,4] (value -0.0)>,
 <gurobi.Var x[3,0] (value -0.0)>,
 <gurobi.Var x[3,1] (value 0.0)>,
 <gurobi.Var x[3,2] (value -0.0)>,
 <gurobi.Var x[3,3] (value 0.0)>,
 <gurobi.Var x[3,4] (value 1.0)>,
 <gurobi.Var x[4,0] (value -0.0)>,
 <gurobi.Var x[4,1] (value -0.0)>,
 <gurobi.Var x[4,2] (value -0.0)>,
 <gurobi.Var x[4,3] (value 1.0)>,
 <gurobi.Var x[4,4] (value -0.0)>,
 <gurobi.Var s[0,0] (value 0.0)>,
 <gurobi.Var s[0,1] (value 13.0)>,
 <gurobi.Var s[0,2] (value 23.0)>,
 <gurobi.Var s[1,0] (value 13.

Abre os dois arquivos que vão ser usados

In [12]:
wb1 = xw.Book("Dupla 09 - Desafio 7 - Instâncias.xlsx")

Roda todos os testes:


Cmax - i=0 - d=2 - T=1e-06 - k=0 = 1199

Cmax - i=0 - d=2 - T=1e-06 - k=1 = 1195

Cmax - i=0 - d=2 - T=1e-06 - k=2 = 1200

Cmax - i=0 - d=2 - T=0.1 - k=0 = 1195

Cmax - i=0 - d=2 - T=0.1 - k=1 = 1199

Cmax - i=0 - d=2 - T=0.1 - k=2 = 1198

Cmax - i=0 - d=2 - T=0.2 - k=0 = 1195

Cmax - i=0 - d=2 - T=0.2 - k=1 = 1199

Cmax - i=0 - d=2 - T=0.2 - k=2 = 1199

Cmax - i=0 - d=2 - T=0.5 - k=0 = 1199

Cmax - i=0 - d=2 - T=0.5 - k=1 = 1198

Cmax - i=0 - d=2 - T=0.5 - k=2 = 1195

Cmax - i=0 - d=3 - T=1e-06 - k=0 = 1199

Cmax - i=0 - d=3 - T=1e-06 - k=1 = 1199

Cmax - i=0 - d=3 - T=1e-06 - k=2 = 1199

Cmax - i=0 - d=3 - T=0.1 - k=0 = 1199

Cmax - i=0 - d=3 - T=0.1 - k=1 = 1199

Cmax - i=0 - d=3 - T=0.1 - k=2 = 1195

Cmax - i=0 - d=3 - T=0.2 - k=0 = 1196

Cmax - i=0 - d=3 - T=0.2 - k=1 = 1198

Cmax - i=0 - d=3 - T=0.2 - k=2 = 1195

Cmax - i=0 - d=3 - T=0.5 - k=0 = 1209

Cmax - i=0 - d=3 - T=0.5 - k=1 = 1196

Cmax - i=0 - d=3 - T=0.5 - k=2 = 1214

Cmax - i=0 - d=5 - T=1e-06 - k=0 = 1199

Cmax - i=0

Fecha o workbook de Dados

In [121]:
wb1.close()